In [2]:
%%capture
!pip install -q tensorflow_text
!pip install -q simpleneighbors[annoy]
!pip install -q nltk
!pip install -q tqdm
!pip3 install tensorflow_text>=2.0.0rc0

In [3]:
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text

In [4]:
# data = pd.read_excel('WHO_FAQ.xlsx')
data = pd.read_csv('covidQ&A.csv')
# Only 450 out of 484 is selected due to limitaion in Hardware
data = data.iloc[:450]

In [5]:
data.head()

,Question,Answere
0,What is COVID-19?,COVID-19 is the disease caused by a new corona...
1,Can adolescents catch COVID-19?,Yes. All age groups can catch COVID-19.While w...
2,Can COVID-19 be passed through breastfeeding?,Transmission of active COVID-19 (virus that ca...
3,Should children wear a mask?,WHO advises that people always consult and abi...
4,What areas should be prioritized for disinfect...,Disinfection practices are important to reduce...


In [6]:
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [17]:
# model
# data.iloc[1]["Answere"]
response_encoding = module.signatures['response_encoder'](
    input=tf.constant(data.Question),
    context=tf.constant(data.Answere)
)['outputs']
# question_encoding = module.signatures['question_encoder'](
#     tf.constant(data.Context)
# )

In [20]:
def ask_question():
    question = str(input("Please ask your question? "))
    return question


def find_answere(question):
    if(question == ""):
        return data.iloc[0].Answere

    question = question.strip()
    question_encoding = module.signatures['question_encoder'](
      tf.constant([question])
  )['outputs']
    similarity = tf.keras.losses.cosine_similarity(response_encoding,question_encoding)
  # print(similarity)
    ans_index = np.argmin(similarity)
  # all_ans_index = np.argsort(similarity)[-3:]
  # print(data.iloc[all_ans_index[2]])
  # print(all_ans_index)
    answere = data.iloc[ans_index].Answere
    return answere

def check_quit(word):
    word = word.strip().lower()
    if(word == "quit"):
        return True
    else:
        return False 

def main():
    print("Type quit to exit the chat")
    while True:
        question = ask_question()
        if(check_quit(question)):
            break
        answere = find_answere(question)
        print(answere)

In [21]:
main()

Type quit to exit the chat
Please ask your question? Is there vaccine for covid
Yes there are now several vaccines that are in use. The first mass vaccination programme started in early December 2020 and the number of vaccination doses administered is updated on a daily basis here.
 At least 13 different vaccines (across 4 platforms) have been administered.The Pfizer/BioNtech Comirnaty vaccine was listed for WHO Emergency Use Listing (EUL) on 31 December 2020. The SII/Covishield and AstraZeneca/AZD1222 vaccines (developed by AstraZeneca/Oxford and manufactured by the State Institute of India and SK Bio
    respectively) were given EUL on 16 February. The Janssen/Ad26.COV 2.S developed by Johnson & Johnson, was listed for EUL on 12 March 2021. The Moderna COVID-19 vaccine (mRNA 1273) was listed for EUL on 30 April 2021 and the Sinopharm COVID-19
    vaccine was listed for EUL on 7 May 2021. The Sinopharm vaccine is produced by Beijing Bio-Institute of Biological Products Co Ltd, subsidi